In [3]:
from keras.models import Sequential, load_model
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
import os

class LeNet:
    def __init__(self, batch_size=32, epochs=20):
        self.batch_size = batch_size
        self.epochs = epochs
        self.model = None
        self._create_lenet()
        self._compile()

    def _create_lenet(self):
    
        self.model = Sequential([
            Conv2D(filters=6, kernel_size=(5, 5), activation='sigmoid',
                   input_shape=(28, 28, 1), padding='same'),
            AveragePooling2D(pool_size=(2, 2), strides=2),
            
            Conv2D(filters=16, kernel_size=(5, 5), activation='sigmoid', padding='same'),
            AveragePooling2D(pool_size=(2, 2), strides=2),
            
            Flatten(),
            Dense(120, activation='sigmoid'),
            Dense(84, activation='sigmoid'),
            Dense(10, activation='softmax')
        ])
    
    def _compile(self):
        """
        Compile the LeNet model with an Adam optimizer and categorical crossentropy loss.
        """
        if self.model is None:
            raise ValueError('Error: Model is not created yet.')
        
        self.model.compile(optimizer='Adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

    def _preprocess(self):
        """
        Preprocess the MNIST dataset: normalize, reshape, and one-hot encode the labels.
        """
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        
        # Normalize the pixel values to the range [0, 1]
        x_train = x_train / 255.0
        x_test = x_test / 255.0
        
        # Add channel dimension
        self.x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
        self.x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
        
        # One-hot encode the labels
        self.y_train = to_categorical(y_train, 10)
        self.y_test = to_categorical(y_test, 10)

    def train(self):
        """
        Train the model using the preprocessed MNIST dataset.
        """
        self._preprocess()
        self.model.fit(self.x_train, self.y_train, 
                       batch_size=self.batch_size, 
                       epochs=self.epochs,
                       validation_data=(self.x_test, self.y_test))

    def save(self, model_path_name):
        """
        Save the trained model to the specified file path.
        """
        if not model_path_name.endswith(".keras"):
            model_path_name += ".keras"
        self.model.save(model_path_name)
        print(f"Model saved as {model_path_name}")

    def load(self, model_path_name):
        """
        Load a saved model from the specified file path.
        """
        if not os.path.exists(model_path_name):
            raise FileNotFoundError(f"No such file: {model_path_name}")
        self.model = load_model(model_path_name)
        print(f"Model loaded from {model_path_name}")

    def predict(self, images):
        """
        Predict the class probabilities for a list of images.
        """
        if self.model is None:
            raise ValueError("Model is not loaded or created yet.")
        
        # Preprocess images: normalize and add channel dimension
        images = [img / 255.0 for img in images]
        images = [img.reshape(1, 28, 28, 1) for img in images]
        
        # Run predictions and return results
        predictions = [self.model.predict(img, verbose=0) for img in images]
        return [pred.argmax() for pred in predictions]

In [4]:
lenet = LeNet(batch_size=64, epochs=10)

In [5]:
lenet.train()

Epoch 1/10
938/938 [==============================] - 12s 5ms/step - loss: 1.0541 - accuracy: 0.6486 - val_loss: 0.3002 - val_accuracy: 0.9148
Epoch 2/10
938/938 [==============================] - 3s 4ms/step - loss: 0.2294 - accuracy: 0.9308 - val_loss: 0.1684 - val_accuracy: 0.9501
Epoch 3/10
938/938 [==============================] - 3s 4ms/step - loss: 0.1509 - accuracy: 0.9546 - val_loss: 0.1306 - val_accuracy: 0.9604
Epoch 4/10
938/938 [==============================] - 3s 4ms/step - loss: 0.1164 - accuracy: 0.9644 - val_loss: 0.1046 - val_accuracy: 0.9674
Epoch 5/10
938/938 [==============================] - 3s 4ms/step - loss: 0.0932 - accuracy: 0.9714 - val_loss: 0.0864 - val_accuracy: 0.9725
Epoch 6/10
938/938 [==============================] - 3s 4ms/step - loss: 0.0791 - accuracy: 0.9757 - val_loss: 0.0670 - val_accuracy: 0.9791
Epoch 7/10
938/938 [==============================] - 3s 4ms/step - loss: 0.0685 - accuracy: 0.9785 - val_loss: 0.0619 - val_accuracy: 0.9799
Epoch

In [6]:
lenet.save("pawar")

Model saved as pawar.keras


In [7]:
lenet.load("pawar.keras")

Model loaded from pawar.keras


In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
x_test.shape

(10000, 28, 28)

In [10]:

type(x_test)

numpy.ndarray

In [11]:
x_test[0:10].shape

(10, 28, 28)

In [12]:
prediction = lenet.predict(x_test[0:10])

In [13]:
prediction

[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]

In [14]:
import numpy as np
import cv2

import os

def process_images_from_folder(folder_path):
    """
    Load and process all images from a folder, ensuring each image is (28, 28) (grayscale).

    Args:
        folder_path (str): Path to the folder containing images.

    Returns:
        list: A list of processed images, each with shape (28, 28).
    """
    processed_images = []
    
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Ensure it's a valid image file
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image = cv2.imread(file_path)  # Load image
            
            if image is not None:  # Check if image is loaded successfully
                # Convert to grayscale (if it's not already)
                grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                
                # Resize to 28x28
                resized_image = cv2.resize(grayscale_image, (28, 28))
                
                # Append to the list
                processed_images.append(resized_image)
    
    return processed_images

In [18]:
folder_path = "H:/MS 2.0/UMDearborn/Courses/Pattern Recognition and neural network/Assignments/Nikhil_Assignments/test_images/" 
test_images = process_images_from_folder(folder_path)

# Predict with LeNet
predictions = lenet.predict(test_images)
print(predictions)

[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 3, 8, 3, 3, 8, 8, 8, 8, 3, 8, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
